In [2]:
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)
os.chdir('../..')

In [3]:
from pathlib import Path

from omegaconf import DictConfig
from hydra import compose, initialize

import numpy as np
import pandas as pd
import torch 

from src.models.recbole import RecboleBench
from src.preprocessing import ClassicDataset
from src.utils.logging import get_logger
from src.utils.processing import data_split, save_results
from src.utils.metrics import run_all_metrics, coverage

logger = get_logger(name=__name__)

%load_ext autoreload
%autoreload 2

In [4]:
# Import recbole Reqired Packages
from recbole.config import Config
from recbole.data import create_dataset, data_preparation, dataset
from recbole.model.general_recommender import BPR, Pop, ItemKNN
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger
from recbole.utils.case_study import full_sort_scores, full_sort_topk
from recbole.quick_start import run_recbole
from recbole.utils import get_model, get_flops, get_trainer

In [5]:
dataset = 'movielens_1m'

In [6]:
# Creat a floder with the name of the dataset
ds_root_path = os.path.join('data', 'tmp')
dataset_path = os.path.join(ds_root_path, dataset)

if not os.path.isdir(dataset_path):
  os.makedirs(dataset_path)
  print("Directory '%s' is created." % dataset_path)
else:
  print("Directory '%s' is exist." % dataset_path)

Directory 'data/tmp/movielens_1m' is exist.


#### Config pre-split dataset
Let’s assume that the dataset is called movielens_1m, and ‍benchmark_filename equals to ['train', 'tune', 'test']. That we will load movielens_1m.train.inter, movielens_1m.tune.inter, movielens_1m.test.inter, and treat them as train, valid, test dataset

In [7]:
# Config pre-spilt dataset
# benchmark-filename as a list of pre-split dataset according to [train dataset, validaition dataset, test dataset]
parameter_dict = {
# dataset config
'data_path': ds_root_path,
'dataset': dataset,
'benchmark_filename': ['train', 'val', 'test'],
'load_col': {'inter': ['user_id', 'item_id', 'rating']},
'neg_sampling': None,
'train_neg_sample_args': None,
'gpu_id': 2,
'metrics': ['Precision', 'Recall', 'MAP', 'NDCG', 'MRR', 'HIT', 'ItemCoverage']
}

In [10]:
# Run recbole
run_recbole(model='Pop', dataset = dataset, config_dict=parameter_dict)

18 Aug 08:43    INFO  ['/home/recsys/miniconda/envs/rec_env/lib/python3.9/site-packages/ipykernel_launcher.py', '--ip=127.0.0.1', '--stdin=9018', '--control=9016', '--hb=9015', '--Session.signature_scheme="hmac-sha256"', '--Session.key=b"8c9304d4-097c-4dbf-91ee-7208587b7769"', '--shell=9017', '--transport="tcp"', '--iopub=9019', '--f=/home/recsys/.local/share/jupyter/runtime/kernel-v2-295WQQpqMq7iKMi.json']
18 Aug 08:43    INFO  
General Hyper Parameters:
gpu_id = 2
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = data/tmp/movielens_1m
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad

{'best_valid_score': 0.154,
 'valid_score_bigger': True,
 'best_valid_result': OrderedDict([('precision@10', 0.0971),
              ('recall@10', 0.0343),
              ('map@10', 0.0535),
              ('ndcg@10', 0.0977),
              ('mrr@10', 0.154),
              ('hit@10', 0.3592),
              ('itemcoverage@10', 0.0269)]),
 'test_result': OrderedDict([('precision@10', 0.1548),
              ('recall@10', 0.0423),
              ('map@10', 0.0857),
              ('ndcg@10', 0.1573),
              ('mrr@10', 0.2666),
              ('hit@10', 0.5568),
              ('itemcoverage@10', 0.0298)])}